In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
from sklearn import preprocessing
from collections import Counter

# from tabulate import tabulate

ICU_FOLDER = "/Users/mai/physionet.org/files/mimiciv/2.0/icu/"
HOSP_FOLDER = "/Users/mai/physionet.org/files/mimiciv/2.0/hosp/"

ROOT_PATH = "/Users/mai/Projects/GitHub/Temporal-MedBERT/"

## MedBERT Pretraining Preparation

In [2]:
patients = pd.read_csv(HOSP_FOLDER + "patients.csv.gz", compression="gzip")
print(patients.shape)
patients.head(5)

(315460, 6)


,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,F,52,2180,2014 - 2016,2180-09-09
1,10000048,F,23,2126,2008 - 2010,NaN
2,10000068,F,19,2160,2008 - 2010,NaN
3,10000084,M,72,2160,2017 - 2019,2161-02-13
4,10000102,F,27,2136,2008 - 2010,NaN


In [3]:
admissions = pd.read_csv(
    HOSP_FOLDER + "admissions.csv.gz",
    compression="gzip",
    parse_dates=["admittime", "dischtime"],
)
print(admissions.shape)
admissions.head(5)

(454324, 15)


,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,NaN,EW EMER.,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-06-26 15:54:00,2180-06-26 21:31:00,0
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,NaN,EW EMER.,EMERGENCY ROOM,HOSPICE,Medicaid,ENGLISH,WIDOWED,WHITE,2180-08-05 20:58:00,2180-08-06 01:44:00,0
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaN,EW EMER.,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-07-23 05:54:00,2180-07-23 14:00:00,0
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,NaN,EU OBSERVATION,EMERGENCY ROOM,NaN,Other,ENGLISH,SINGLE,WHITE,2160-03-03 21:55:00,2160-03-04 06:26:00,0


In [4]:
admissions["admittime"] = pd.to_datetime(admissions["admittime"]).dt.date
admissions["dischtime"] = pd.to_datetime(admissions["dischtime"]).dt.date
admissions = admissions[["subject_id", "hadm_id", "admittime", "dischtime"]]
admissions.head(5)


,subject_id,hadm_id,admittime,dischtime
0,10000032,22595853,2180-05-06,2180-05-07
1,10000032,22841357,2180-06-26,2180-06-27
2,10000032,25742920,2180-08-05,2180-08-07
3,10000032,29079034,2180-07-23,2180-07-25
4,10000068,25022803,2160-03-03,2160-03-04


In [5]:
diagnosis = pd.read_csv(HOSP_FOLDER + "diagnoses_icd.csv.gz", compression="gzip")
diagnosis.head(5)
print(len(np.unique(diagnosis["subject_id"])))

190179


In [6]:
np.unique(diagnosis["seq_num"])


array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39])

In [7]:
diagnosis["diagnosis"] = (
    "ICD"
    + diagnosis["icd_version"].astype(str)
    + "_"
    + diagnosis["icd_code"].astype(str)
)
diagnosis = diagnosis.drop(columns=["icd_code", "icd_version"])
diagnosis.head(5)

,subject_id,hadm_id,seq_num,diagnosis
0,10000032,22595853,1,ICD9_5723
1,10000032,22595853,2,ICD9_78959
2,10000032,22595853,3,ICD9_5715
3,10000032,22595853,4,ICD9_07070
4,10000032,22595853,5,ICD9_496


In [8]:
procedures = pd.read_csv(HOSP_FOLDER + "procedures_icd.csv.gz", compression="gzip")
procedures.head()

,subject_id,hadm_id,seq_num,chartdate,icd_code,icd_version
0,10000032,22595853,1,2180-05-07,5491,9
1,10000032,22841357,1,2180-06-27,5491,9
2,10000032,25742920,1,2180-08-06,5491,9
3,10000068,25022803,1,2160-03-03,8938,9
4,10000117,27988844,1,2183-09-19,0QS734Z,10


In [9]:
procedure_number = []
for i in range(procedures.shape[0]):
    procedure_number.append(
        (procedures.hadm_id.values == procedures.loc[i, "hadm_id"]).sum()
    )

procedures["num_prod"] = procedure_number
print("Maximum number of procedures:", max(procedure_number))
print("Minimum number of procedures:", min(procedure_number))

Maximum number of procedures: 41
Minimum number of procedures: 1


In [10]:
procedures = procedures.drop(
    columns=["seq_num", "chartdate", "icd_code", "icd_version"]
)
procedures.head(5)

,subject_id,hadm_id,num_prod
0,10000032,22595853,1
1,10000032,22841357,1
2,10000032,25742920,1
3,10000068,25022803,1
4,10000117,27988844,1


In [11]:
print("admissions size", admissions.shape)
print("diagnosis size:", diagnosis.shape)
print("procedures size", procedures.shape)


admissions size (454324, 4)
diagnosis size: (5006884, 4)
procedures size (704124, 3)


In [12]:
trial_1 = pd.merge(admissions, diagnosis, on=["subject_id", "hadm_id"], how="right")
trial_1

,subject_id,hadm_id,admittime,dischtime,seq_num,diagnosis
0,10000032,22595853,2180-05-06,2180-05-07,1,ICD9_5723
1,10000032,22595853,2180-05-06,2180-05-07,2,ICD9_78959
2,10000032,22595853,2180-05-06,2180-05-07,3,ICD9_5715
3,10000032,22595853,2180-05-06,2180-05-07,4,ICD9_07070
4,10000032,22595853,2180-05-06,2180-05-07,5,ICD9_496
...,...,...,...,...,...,...
5006879,19999987,23865745,2145-11-02,2145-11-11,7,ICD9_41401
5006880,19999987,23865745,2145-11-02,2145-11-11,8,ICD9_78039
5006881,19999987,23865745,2145-11-02,2145-11-11,9,ICD9_0413
5006882,19999987,23865745,2145-11-02,2145-11-11,10,ICD9_36846


In [13]:
trial_2 = pd.merge(trial_1, procedures, on=["subject_id", "hadm_id"], how="inner")
trial_2

,subject_id,hadm_id,admittime,dischtime,seq_num,diagnosis,num_prod
0,10000032,22595853,2180-05-06,2180-05-07,1,ICD9_5723,1
1,10000032,22595853,2180-05-06,2180-05-07,2,ICD9_78959,1
2,10000032,22595853,2180-05-06,2180-05-07,3,ICD9_5715,1
3,10000032,22595853,2180-05-06,2180-05-07,4,ICD9_07070,1
4,10000032,22595853,2180-05-06,2180-05-07,5,ICD9_496,1
...,...,...,...,...,...,...,...
10241876,19999987,23865745,2145-11-02,2145-11-11,9,ICD9_0413,2
10241877,19999987,23865745,2145-11-02,2145-11-11,10,ICD9_36846,2
10241878,19999987,23865745,2145-11-02,2145-11-11,10,ICD9_36846,2
10241879,19999987,23865745,2145-11-02,2145-11-11,11,ICD9_7810,2


In [14]:
cols = ["subject_id", "admittime", "dischtime",
        "diagnosis", "seq_num", "num_prod"]
data = trial_2[cols]


In [15]:
data


,subject_id,admittime,dischtime,diagnosis,seq_num,num_prod
0,10000032,2180-05-06,2180-05-07,ICD9_5723,1,1
1,10000032,2180-05-06,2180-05-07,ICD9_78959,2,1
2,10000032,2180-05-06,2180-05-07,ICD9_5715,3,1
3,10000032,2180-05-06,2180-05-07,ICD9_07070,4,1
4,10000032,2180-05-06,2180-05-07,ICD9_496,5,1
...,...,...,...,...,...,...
10241876,19999987,2145-11-02,2145-11-11,ICD9_0413,9,2
10241877,19999987,2145-11-02,2145-11-11,ICD9_36846,10,2
10241878,19999987,2145-11-02,2145-11-11,ICD9_36846,10,2
10241879,19999987,2145-11-02,2145-11-11,ICD9_7810,11,2


In [16]:
# df.to_csv("Data_MedBert.csv", sep="\t", index=False)

In [17]:
# data = pd.read_csv("Data_MedBert.csv", sep="\t")
# data.head(5)

In [18]:
len(np.unique(data["subject_id"]))

128256

In [19]:
remove_dup_ad = data.drop_duplicates(["subject_id", "admittime"])
remove_dup_ad.head(5)

,subject_id,admittime,dischtime,diagnosis,seq_num,num_prod
0,10000032,2180-05-06,2180-05-07,ICD9_5723,1,1
8,10000032,2180-06-26,2180-06-27,ICD9_07071,1,1
16,10000032,2180-08-05,2180-08-07,ICD9_07054,1,1
26,10000068,2160-03-03,2160-03-04,ICD9_30500,1,1
27,10000117,2183-09-18,2183-09-21,ICD10_S72012A,1,1


In [20]:
x = Counter(remove_dup_ad["subject_id"])
x.most_common()


[(13475033, 70),
 (11296936, 66),
 (12251785, 64),
 (12596559, 64),
 (11761621, 52),
 (17200404, 49),
 (17716210, 49),
 (18553055, 49),
 (10827966, 48),
 (13297743, 48),
 (18965447, 48),
 (18376342, 47),
 (10998537, 45),
 (17477304, 45),
 (18520744, 45),
 (18194969, 44),
 (10123949, 43),
 (11826927, 42),
 (11965254, 42),
 (12468016, 41),
 (12985376, 41),
 (17937834, 41),
 (18358138, 40),
 (15884728, 38),
 (15942934, 38),
 (12324099, 37),
 (13251065, 37),
 (15464144, 37),
 (16339049, 37),
 (12406461, 36),
 (15987325, 36),
 (17340686, 36),
 (18655830, 36),
 (18676703, 36),
 (15197756, 35),
 (17517983, 34),
 (11281568, 33),
 (12547294, 33),
 (14061397, 33),
 (14464902, 33),
 (18207287, 33),
 (12882985, 32),
 (13813803, 32),
 (15179083, 32),
 (18284271, 32),
 (19093092, 32),
 (10213338, 31),
 (13273041, 31),
 (13470788, 31),
 (14345906, 31),
 (14841168, 31),
 (15455517, 31),
 (15783916, 31),
 (16439884, 31),
 (11236474, 30),
 (16073325, 30),
 (16909817, 30),
 (17295976, 30),
 (11786671, 29

## Finetuning MedBERT Data Preparation

In [21]:
patients = pd.read_csv(HOSP_FOLDER + "patients.csv.gz", compression="gzip")
print(patients.shape)
patients.head(5)

(315460, 6)


,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,F,52,2180,2014 - 2016,2180-09-09
1,10000048,F,23,2126,2008 - 2010,NaN
2,10000068,F,19,2160,2008 - 2010,NaN
3,10000084,M,72,2160,2017 - 2019,2161-02-13
4,10000102,F,27,2136,2008 - 2010,NaN


In [22]:
admissions = pd.read_csv(
    HOSP_FOLDER + "admissions.csv.gz",
    compression="gzip",
    parse_dates=["admittime", "dischtime"],
)
print(admissions.shape)
admissions["admittime"] = pd.to_datetime(admissions["admittime"]).dt.date
admissions["dischtime"] = pd.to_datetime(admissions["dischtime"]).dt.date
admissions = admissions[
    ["subject_id", "hadm_id", "admittime", "dischtime", "hospital_expire_flag"]
]
admissions.head(5)

(454324, 15)


,subject_id,hadm_id,admittime,dischtime,hospital_expire_flag
0,10000032,22595853,2180-05-06,2180-05-07,0
1,10000032,22841357,2180-06-26,2180-06-27,0
2,10000032,25742920,2180-08-05,2180-08-07,0
3,10000032,29079034,2180-07-23,2180-07-25,0
4,10000068,25022803,2160-03-03,2160-03-04,0


In [23]:
np.unique(admissions["hospital_expire_flag"])

array([0, 1])

In [24]:
admissions["hospital_expire_flag"].value_counts()

0    445224
1      9100
Name: hospital_expire_flag, dtype: int64

In [25]:
diagnosis = pd.read_csv(HOSP_FOLDER + "diagnoses_icd.csv.gz", compression="gzip")
diagnosis.head(5)
print(len(np.unique(diagnosis["subject_id"])))

190179


In [26]:
diagnosis["diagnosis"] = (
    "ICD"
    + diagnosis["icd_version"].astype(str)
    + "_"
    + diagnosis["icd_code"].astype(str)
)
diagnosis = diagnosis.drop(columns=["icd_code", "icd_version"])
diagnosis.head(5)

,subject_id,hadm_id,seq_num,diagnosis
0,10000032,22595853,1,ICD9_5723
1,10000032,22595853,2,ICD9_78959
2,10000032,22595853,3,ICD9_5715
3,10000032,22595853,4,ICD9_07070
4,10000032,22595853,5,ICD9_496


In [27]:
procedures = pd.read_csv(HOSP_FOLDER + "procedures_icd.csv.gz", compression="gzip")
procedure_number = []
for i in range(procedures.shape[0]):
    procedure_number.append(
        (procedures.hadm_id.values == procedures.loc[i, "hadm_id"]).sum()
    )

procedures["num_prod"] = procedure_number
print("Maximum number of procedures:", max(procedure_number))
print("Minimum number of procedures:", min(procedure_number))
procedures.head()

Maximum number of procedures: 41
Minimum number of procedures: 1


,subject_id,hadm_id,seq_num,chartdate,icd_code,icd_version,num_prod
0,10000032,22595853,1,2180-05-07,5491,9,1
1,10000032,22841357,1,2180-06-27,5491,9,1
2,10000032,25742920,1,2180-08-06,5491,9,1
3,10000068,25022803,1,2160-03-03,8938,9,1
4,10000117,27988844,1,2183-09-19,0QS734Z,10,1


In [28]:
procedures = procedures.drop(
    columns=["seq_num", "chartdate", "icd_code", "icd_version"]
)
procedures.head(5)

,subject_id,hadm_id,num_prod
0,10000032,22595853,1
1,10000032,22841357,1
2,10000032,25742920,1
3,10000068,25022803,1
4,10000117,27988844,1


In [29]:
# micro = pd.read_csv(HOSP_FOLDER + "microbiologyevents.csv.gz", compression="gzip")
# micro = micro[micro['interpretation'].notna()]
# micro = micro[(micro["interpretation"] != "P") & (micro["interpretation"] != "I")]
# micro = micro.reset_index(drop = True)

# label_encoder = preprocessing.LabelEncoder()
# micro["interpretation"] = label_encoder.fit_transform(micro["interpretation"])
# micro = micro[micro["ab_name"] == "GENTAMICIN"].reset_index(drop=True)
# # micro = micro[["subject_id", "hadm_id", "interpretation"]]
# print("New shape, dataset of antibiotics:", micro.shape, "\n")
# print("Number of patients:", len(np.unique(micro["subject_id"])), "\n")
# print("Number of admissions:", len(np.unique(micro["hadm_id"])), "\n")

# micro.head()

In [30]:
# micro = micro[["subject_id", "hadm_id", "interpretation"]]


In [31]:
# micro_admin = micro.dropna().reset_index(drop=True)
# micro_admin.shape


In [32]:
# micro_admin.head(5)


In [33]:
# print("New shape, dataset of antibiotics:", micro_admin.shape, "\n")
# print("Number of patients:", len(np.unique(micro_admin["subject_id"])), "\n")
# print("Number of admissions:", len(np.unique(micro_admin["hadm_id"])), "\n")


In [34]:
print("patients size:", patients.shape)
print("admission size:", admissions.shape)
print("diagnosis size:", diagnosis.shape)
print("procedures size:", procedures.shape)
# print("microbiology size:", micro_admin.shape)


patients size: (315460, 6)
admission size: (454324, 5)
diagnosis size: (5006884, 4)
procedures size: (704124, 3)


In [35]:
trial_0 = admissions.merge(diagnosis, on=["subject_id", "hadm_id"])
trial_0.shape

(5006884, 7)

In [36]:
trial_1 = pd.merge(trial_0, procedures, on=["subject_id", "hadm_id"], how="inner")
print("Shape", trial_1.shape, "\n")
trial_1.head(5)

Shape (10241881, 8) 



,subject_id,hadm_id,admittime,dischtime,hospital_expire_flag,seq_num,diagnosis,num_prod
0,10000032,22595853,2180-05-06,2180-05-07,0,1,ICD9_5723,1
1,10000032,22595853,2180-05-06,2180-05-07,0,2,ICD9_78959,1
2,10000032,22595853,2180-05-06,2180-05-07,0,3,ICD9_5715,1
3,10000032,22595853,2180-05-06,2180-05-07,0,4,ICD9_07070,1
4,10000032,22595853,2180-05-06,2180-05-07,0,5,ICD9_496,1


In [37]:
trial_1 = trial_1.drop_duplicates().reset_index(drop=True)
print("Shape", trial_1.shape, "\n")


Shape (2963978, 8) 



In [38]:
# trial_2 = pd.merge(trial_1, micro_admin, on=['subject_id', "hadm_id"], how = "inner")
# trial_2


In [39]:
trial_1 = trial_1.drop_duplicates()
print("Shape of finetuning dataset:", trial_1.shape)
print("Number of patients:", len(np.unique(trial_1["subject_id"])), "\n")
print("Number of admissions:", len(np.unique(trial_1["hadm_id"])), "\n")


Shape of finetuning dataset: (2963978, 8)
Number of patients: 128256 

Number of admissions: 241334 



In [40]:
cols = [
    "subject_id",
    "admittime",
    "dischtime",
    "hospital_expire_flag",
    "diagnosis",
    "seq_num",
    "num_prod",
]
df = trial_1[cols]

In [41]:
df.to_csv(
    ROOT_PATH + "data/finetuning_mortality/mortality_finetunining_dataset.csv",
    sep="\t",
    index=False,
)

In [42]:
df.head(5)


,subject_id,admittime,dischtime,hospital_expire_flag,diagnosis,seq_num,num_prod
0,10000032,2180-05-06,2180-05-07,0,ICD9_5723,1,1
1,10000032,2180-05-06,2180-05-07,0,ICD9_78959,2,1
2,10000032,2180-05-06,2180-05-07,0,ICD9_5715,3,1
3,10000032,2180-05-06,2180-05-07,0,ICD9_07070,4,1
4,10000032,2180-05-06,2180-05-07,0,ICD9_496,5,1


In [43]:
df["hospital_expire_flag"].value_counts()

0    2796577
1     167401
Name: hospital_expire_flag, dtype: int64